# Aggregate urban rooftops and generate kml outputs

In [ ]:
import os
from pathlib import Path

import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

from rooftop_tools.utils_rooftop import get_nearest_points_on_road

In [ ]:
# set paths
FH_SAMPLING_FOLDER = Path("../") / "data" / "fortify_data"
SAMPLES_FOLDER = FH_SAMPLING_FOLDER / "Samples"

## 1. Import excel files with inspected points and create single df of urban roooftops

In [ ]:
all_sampled_rooftops = pd.read_excel(SAMPLES_FOLDER / "Combined_sample.xlsx")
urban_rooftops = all_sampled_rooftops[all_sampled_rooftops["is urban?"] == "Y"]

geom = gpd.GeoSeries.from_wkt(urban_rooftops["geometry"])
urban_rooftops = gpd.GeoDataFrame(
    urban_rooftops.drop(columns=["geometry"]), geometry=geom, crs="EPSG:4326"
)

print(f"Number of urban rooftops: {len(urban_rooftops)}")
urban_rooftops.head()

## 2. Get nearest point on road

In [ ]:
api_key = os.environ["GOOGLE_MAPS_API_KEY"]
urban_rooftops["nearest_point_on_road"] = get_nearest_points_on_road(
    urban_rooftops, api_key
)

num_points_off_road = len(
    urban_rooftops[urban_rooftops["nearest_point_on_road"].isna()]
)
print(
    f"Number points off road: {num_points_off_road}, total points: {len(urban_rooftops)}"
)

## 3. Save csv with google map links

In [ ]:
temp_samp_points = urban_rooftops[
    ["d_name", "row_num", "Centre name", "geometry", "nearest_point_on_road"]
]
temp_samp_points["orig_lat"] = temp_samp_points["geometry"].y
temp_samp_points["orig_lon"] = temp_samp_points["geometry"].x
temp_samp_points["road_lat"] = temp_samp_points["nearest_point_on_road"].y
temp_samp_points["road_lon"] = temp_samp_points["nearest_point_on_road"].x
temp_samp_points = temp_samp_points.drop(columns=["geometry", "nearest_point_on_road"])
temp_samp_points["google_maps_directions_link"] = temp_samp_points.apply(
    lambda row: f"https://www.google.com/maps/dir/{row['orig_lat']},{row['orig_lon']}/{row['road_lat']},{row['road_lon']}",
    axis=1,
)
# replace google_maps_directions_link with a link to just the original point if nearest_point_on_road is null
temp_samp_points.loc[
    temp_samp_points["road_lat"].isna(), "google_maps_directions_link"
] = temp_samp_points.loc[
    temp_samp_points["road_lat"].isna(), ["orig_lat", "orig_lon"]
].apply(
    lambda x: f"https://www.google.com/maps?q={x[0]},{x[1]}", axis=1
)


# save the sampled points as csv
temp_samp_points.to_csv(SAMPLES_FOLDER / f"Fortify_sampled_rooftops.csv", index=False)

## 4. Save as kmls

In [ ]:
urban_rooftops.head()

In [ ]:
sampled_points_on_road = urban_rooftops[
    ~(urban_rooftops["nearest_point_on_road"].isnull())
]
sampled_points_off_road = urban_rooftops[
    urban_rooftops["nearest_point_on_road"].isnull()
]

# save lines between original points and nearest points on road
sampled_points_on_road["line"] = sampled_points_on_road.apply(
    lambda row: LineString([row["geometry"], row["nearest_point_on_road"]]), axis=1
)
lines_gdf = sampled_points_on_road[["line", "row_num"]]
lines_gdf.to_file(SAMPLES_FOLDER / "lines.kml", driver="KML")

# save sampled points on road as kml
sampled_points_on_road[["nearest_point_on_road", "row_num"]].to_file(
    SAMPLES_FOLDER / "points_on_road`.kml", driver="KML"
)

# save sampled points off road as kml
sampled_points_off_road[["geometry", "row_num"]].to_file(
    SAMPLES_FOLDER / "points_off_road.kml", driver="KML"
)